## Testing

In [ ]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the application code from Day 3 to provide context for test generation
# Fix: Load directly from app/main.py instead of using load_artifact which looks in artifacts directory
app_main_path = os.path.join(project_root, "app", "main.py")
try:
    with open(app_main_path, 'r', encoding='utf-8') as f:
        app_code = f.read()
    print(f"Successfully loaded app code from: {app_main_path}")
except FileNotFoundError:
    app_code = None
    print(f"Warning: Could not load app/main.py from {app_main_path}. Lab may not function correctly.")
except Exception as e:
    app_code = None
    print(f"Error loading app/main.py: {e}")

In [ ]:
# TODO: Write a prompt to generate happy path tests for your API.
happy_path_tests_prompt = f"""
REPLACE
"""

print("--- Generating Happy Path Tests ---")
if app_code:
    generated_happy_path_tests = get_completion(happy_path_tests_prompt, client, model_name, api_provider)
    cleaned_tests = clean_llm_output(generated_happy_path_tests, language='python')
    print(cleaned_tests)
    save_artifact(cleaned_tests, "tests/test_main_simple.py")
else:
    print("Skipping test generation because app code is missing.")

In [ ]:
# TODO: Write a prompt to generate edge case tests.
edge_case_tests_prompt = f"""
REPLACE
"""

print("--- Generating Edge Case Tests ---")
if app_code:
    generated_edge_case_tests = get_completion(edge_case_tests_prompt, client, model_name, api_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
else:
    print("Skipping test generation because app code is missing.")

In [ ]:
# TODO: Write a prompt to generate the pytest fixture for an isolated test database.
db_fixture_prompt = f"""
REPLACE
"""

print("--- Generating Pytest DB Fixture ---")
if app_code:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    save_artifact(cleaned_fixture, "tests/conftest.py", overwrite=True)
else:
    print("Skipping fixture generation because app context is missing.")

# TODO: Write a prompt to refactor the happy path tests to use the new fixture.
refactor_tests_prompt = f"""
You are a QA Engineer. Given the FastAPI application source below and the tests/conftest.py fixture that provides a pytest `client` fixture (TestClient configured to use an in-memory DB), generate a single Python file (no extra commentary) named tests/test_main_with_fixture.py.

Requirements for tests/test_main_with_fixture.py:
- Do NOT create a TestClient or import the app; rely on the `client` fixture (function parameter) provided by tests/conftest.py.
- Import only what is necessary (e.g., time or datetime for generating unique emails).
- Include two pytest test functions that use the `client` fixture:
    1. test_create_user_happy_path(client):
        - POST /users/ with a valid payload containing first_name, last_name, email, start_date.
        - Assert response.status_code == 201
        - Assert response.json() contains 'user_id', 'email', and 'full_name'
        - Assert the returned 'email' matches the payload
    2. test_list_users_happy_path(client):
        - Ensure the test creates a user inside the test (POST /users/) to make the test independent.
        - GET /users/ and assert response.status_code == 200
        - Assert the response is a list and contains at least one user
        - Assert the list contains the email created in this test
- Use a timestamp-based unique email in each test so they remain independent even when run together.
- Keep tests self-contained and runnable with pytest (include necessary imports).
- Do not include explanations or non-code text — output only the Python source for tests/test_main_with_fixture.py.

Application source for context (do not modify it; use to ensure correct field names and endpoints):
{app_code}
"""

print("\n--- Generating Refactored Tests ---")
if app_code:
    refactored_tests = get_completion(refactor_tests_prompt, client, model_name, api_provider)
    cleaned_refactored_tests = clean_llm_output(refactored_tests, language='python')
    print(cleaned_refactored_tests)
    save_artifact(cleaned_refactored_tests, "tests/test_main_with_fixture.py", overwrite=True)
else:
    print("Skipping test refactoring because app context is missing.")

print("\n--- Generating Refactored Tests ---")
if app_code:
    refactored_tests = get_completion(refactor_tests_prompt, client, model_name, api_provider)
    cleaned_refactored_tests = clean_llm_output(refactored_tests, language='python')
    print(cleaned_refactored_tests)
    save_artifact(cleaned_refactored_tests, "tests/test_main_with_fixture.py", overwrite=True)
else:
    print("Skipping test refactoring because app context is missing.")

**iNJECTING BUGGY CODE?**


In [ ]:
# --- Task 1: Analyze and explain the bug --- 
analyze_bug_prompt = f"""
You are a senior Python developer acting as a code reviewer.
Analyze the following Python function and explain the logical error in it. Be concise.

```python
{buggy_code}
```
"""
print("--- Analyzing Bug ---")
bug_explanation = get_completion(analyze_bug_prompt, client, model_name, api_provider)
print(bug_explanation)

# --- Task 2: Write a failing pytest test ---
failing_test_prompt = f"""
You are a QA engineer writing tests with pytest.
Based on the following Python function, write a test function named `test_calculate_total_with_discount` that will FAIL because of the bug in the discount calculation. The test should assert the correct expected value.

```python
{buggy_code}
```
"""
print("\n--- Generating Failing Test ---")
failing_test_code_raw = get_completion(failing_test_prompt, client, model_name, api_provider)
failing_test_code = clean_llm_output(failing_test_code_raw, language='python')
print(failing_test_code)

# --- Task 3: Fix the code based on the failing test ---
fix_code_prompt = f"""
You are a senior Python developer tasked with fixing a bug.

The following function is buggy:
<buggy_code>
```python
{buggy_code}
```
</buggy_code>

The following pytest test fails when run against it, demonstrating the bug:
<failing_test>
```python
{failing_test_code}
```
</failing_test>

Your task is to fix the `calculate_total` function so that the provided test will pass. Output only the corrected function.
"""
print("\n--- Generating Fixed Code ---")
fixed_code_raw = get_completion(fix_code_prompt, client, model_name, api_provider)
fixed_code = clean_llm_output(fixed_code_raw, language='python')
print(fixed_code)

# Save the final corrected code
if fixed_code:
    save_artifact(fixed_code, "app/day4_sp_fixed_shopping_cart.py")